En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [17]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda


import pandas as pd
import requests
from datetime import datetime, timedelta
import ast

Definimos la clase y las variables:

In [18]:
class Extraccion_clima:
     def __init__(self, diccionario):
        self.diccionario = diccionario
        

Y ahora vamos con los métodos:
1. Funcion para llamar a la Api y extraer datos

In [19]:
class Extraccion_clima:
    def __init__(self, diccionario):
        self.diccionario = diccionario  

#definimos funcion para llamar a la api y sacar los datos.    
    def llamada_extraccion (self, diccionario):
        df_clima=pd.DataFrame()
        for k, v in diccionario.items():
            
            url= f'http://www.7timer.info/bin/api.pl?lon={v[0]}&lat={v[1]}&product=meteo&output=json'
            response= requests.get (url)
            df=pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
            df['pais']= k
            df_clima= pd.concat([df_clima, df])

            
            if response.status_code == 200:
                print(response.status_code,':la peticion se ha realizado correctamente:',response.reason)
            elif response.status_code == 402:
                print(response.status_code, ':No se ha podido autorizar:',response.reason)
            elif response.status_code == 404:
                print(response.status_code,':Algo ha salido mal, el recurso no se ha encontrado:',response.reason)
            else:
                print(response.status_code,':Algo inesperado ha ocurrido:',response.reason)

            df_clima.to_csv('datos/df_clima.csv')   
            df_clima.to_pickle('datos/df_clima.pkl') 
            return df_clima


#definimos funcion para limpieza del df resultante
    def limpieza_clima (self, df_clima1):
        self.df_clima = df_clima1
        
    #1.para sacar columnas rh
        
        #df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)
        y_rh = df_clima1['rh_profile'].apply(pd.Series) 
        
        for i in range(len(y_rh.columns)): 
            nombre = "rh_" + str(y_rh[i].apply(pd.Series)["layer"][0]) 
                
            valores = list(y_rh[i].apply(pd.Series)["rh"] )
        
            df_clima1.insert(i, nombre, valores)

    #2.para wind_profile, la columna direccion
        
        #df_clima['wind_profile']=df_clima['wind_profile'].apply(ast.literal_eval)
        x_wind = df_clima1['wind_profile'].apply(pd.Series)

        for i in range(len(x_wind.columns)): 

            nombre = "direction" + str(x_wind[i].apply(pd.Series)["layer"][0]) 

            valores = list(x_wind[i].apply(pd.Series)["direction"] )

            df_clima1.insert(i, nombre, valores)

    #3.para wind_profile, la columna speed
        
        for i in range(len(x_wind.columns)): 
        
            nombre = "speed" + str(x_wind[i].apply(pd.Series)["layer"][0]) 
    
            valores = list(x_wind[i].apply(pd.Series)["speed"] )

            df_clima1.insert(i, nombre, valores)
            
        return df_clima1
        
#definimos funcion para la union del df anterior con el de tiburones
    def union_clima_tiburones (self, df_clima2):

        self.df_clima2 = df_clima2
        df_attacks= pd.read_csv ('datos/df_attacks_limpieza_completa.csv', index_col=0)
   
#pasamos los nombres de los paises a minusculas para asegurarnos que se puede hacer un mergeo    
        df_clima2['pais'] = df_clima2['pais'].apply (lambda x : x.lower() )

#agrupamos el df clima por paises y le pedimos la media de las variables
        df_clima_agrupado = df_clima2.groupby (['pais']).mean()

#filtramos df attacks para quedarnos con los paises que queremos
        df_attacks2 = df_attacks[df_attacks['country'].isin (['usa','australia', 'papua new guinea','new zealand','south africa'])]

#juntamos
        df_clima_ataques=pd.merge(df_attacks2, df_clima_agrupado, right_on='pais', left_on='country')    

#guardamos    
        df_clima_ataques.to_csv('datos/df_clima_ataques.csv')
        df_clima_ataques.to_pickle('datos/df_clima_ataques.pkl')

        return df_clima_ataques


Comprobamos

In [20]:
diccionario_prueba={'USA':['-100.445882', '39.7837304'], 'Australia':['134.755', '-24.7761086'], 'South Africa':['24.991639', '-28.8166236'],'New Zealand': ['172.8344077', '-41.5000831'], 'Papua New Guinea': ['144.2489081','-5.6816069' ] }

In [21]:
api = Extraccion_clima(diccionario_prueba)

In [22]:
df_extraccion= api.llamada_extraccion(diccionario_prueba)


200 :la peticion se ha realizado correctamente: OK


In [23]:
df_extraccion.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,USA


In [24]:
df_limpieza_clima=api.limpieza_clima (df_extraccion)